# neandertools tutorial

This notebook demonstrates the two main ways to use **neandertools** on the Rubin Science Platform:

1. **Low-level** — extract cutouts from known `(visit, detector)` pairs directly via `cutouts_from_butler`
2. **High-level** — run the full asteroid tracking pipeline with `AsteroidCutoutPipeline`

Both require an active connection to the Butler repository and the LSST software stack.

## Setup

In [ ]:
# If neandertools is not yet installed in your RSP environment:
# !pip install -e /path/to/neandertools --quiet

import matplotlib.pyplot as plt
from IPython.display import Image, display

import neandertools as nt
from neandertools import AsteroidCutoutPipeline, cutouts_from_butler

---
## Part 1 — Low-level cutout extraction

`cutouts_from_butler` gives you direct access to image cutouts when you already know the `visit` and `detector` IDs.
This is the fastest path if you are exploring a single image or building your own pipeline on top.

In [ ]:
# Connect to the Butler repository
svc = cutouts_from_butler(
    "dp1",
    collections=["LSSTComCam/DP1"],
)

### 1a — Extract by pixel coordinates

In [ ]:
# Returns a list with one cutout (an LSST ExposureF object)
images = svc.cutout(
    visit=2024110800253,
    detector=5,
    x=2036,
    y=2000,
    h=201,
    w=201,
)
cutout = images[0]
print(f"Cutout shape: {cutout.image.array.shape}")

### 1b — Extract by sky coordinates

When you provide `ra` and `dec` instead of `x` and `y`, the pixel centre is resolved
through the image WCS automatically.

In [ ]:
images = svc.cutout(
    visit=2024110800253,
    detector=5,
    ra=62.1,
    dec=-31.2,
    h=201,
    w=201,
)
cutout = images[0]

plt.figure(figsize=(5, 5))
plt.imshow(cutout.image.array, origin="lower", cmap="gray",
           vmin=0, vmax=500)
plt.colorbar(label="Flux")
plt.title("Single cutout (ra/dec centre)")
plt.tight_layout()
plt.show()

### 1c — Vectorised extraction

All arguments accept arrays.  Each row is an independent cutout request.

In [ ]:
images = svc.cutout(
    visit=[2024110800253, 2024110800253],
    detector=[5, 5],
    ra=[62.1, 62.2],
    dec=[-31.2, -31.3],
    h=151,
    w=151,
)
print(f"Got {len(images)} cutouts")

### 1d — Spatial–temporal lookup

`find_visit_detector` searches which `(visit, detector)` pairs cover a sky position at a given time.
Useful when you know the sky position but not the data IDs.

In [ ]:
visits, detectors = svc.find_visit_detector(
    ra=53.0,
    dec=-27.91,
    t="2024-11-09T06:12:10",
)
print(f"Found {len(visits)} matching (visit, detector) pair(s):")
for v, d in zip(visits, detectors):
    print(f"  visit={v}  detector={d}")

---
## Part 2 — Full asteroid tracking pipeline

`AsteroidCutoutPipeline` automates the complete workflow:

1. Query JPL Horizons for the ephemeris
2. Build sky-search polygons along the predicted track
3. Find all overlapping Butler images
4. Extract a cutout centred on the interpolated asteroid position at each epoch
5. Render an animated GIF with the asteroid fixed at the centre

Frames where the target falls on a trimmed image border are automatically discarded.

### 2a — Create and run the pipeline

In [ ]:
pipeline = AsteroidCutoutPipeline(
    target="1991 SJ",
    start="2024-11-25",
    end="2024-11-28",
    dr="dp1",
    collection="LSSTComCam/DP1",
    cutout_size=200,
    step="12h",
)

In [ ]:
gif_path = pipeline.run(
    output_path="1991SJ.gif",
    warp_common_grid=True,   # align all frames on a common sky grid
    show_ne=True,            # draw a North/East compass indicator
    match_background=True,   # subtract per-frame sky background
    match_noise=True,        # divide by per-frame RMS (SNR display)
    frame_duration_ms=500,
)
print(f"GIF written to: {gif_path}")
print(f"Frames kept: {len(pipeline.cutouts)}")

### 2b — Display the GIF inline

In [ ]:
display(Image(filename=str(gif_path)))

### 2c — Contact-sheet grid

`pipeline.grid()` displays all frames side-by-side using the same display options as the GIF.
Call it after `run()` — it reuses the already-extracted cutouts.

In [ ]:
fig, axes = pipeline.grid(
    ncols=5,
    warp_common_grid=True,
    show_ne=True,
    match_background=True,
    match_noise=True,
)
# Save the grid separately if needed
fig.savefig("1991SJ_grid.png", dpi=150, bbox_inches="tight")

---
## Part 3 — Tips

### Equivalent command-line call

The pipeline above is identical to:

```bash
python run_pipeline.py "1991 SJ" 2024-11-25 2024-11-28 \
    --output 1991SJ.gif --warp --grid --cutout-size 200 \
    --match-noise --show-ne --step 12h
```

The CLI is the recommended option when the notebook kernel is unstable.

### Choosing parameters

| Parameter | Guidance |
|---|---|
| `cutout_size` | 100–200 px works well for most main-belt objects. Increase for fast-moving NEOs. |
| `step` | `12h` is usually sufficient. Use `1h` only if you suspect the ephemeris grid is too coarse for the object's motion. |
| `warp_common_grid` | Recommended for GIFs — aligns frames so the asteroid is pinned at centre and stars drift. Requires LSST warp modules. |
| `match_background` | Almost always on. Removes sky background differences between epochs. |
| `match_noise` | Useful for very faint targets — normalises each frame to the same noise level. |
| `polygon_widening_arcsec` | Default 2 arcsec is conservative. Increase if images are missed; decrease to reduce false-positive candidates. |

### Inspecting intermediate results

After `run()` the pipeline object retains all intermediate data:

In [ ]:
# Ephemeris (from JPL Horizons)
eph = pipeline.ephemeris
print("Ephemeris points:", len(eph["ra_deg"]))
print("RA range: {:.4f} – {:.4f} deg".format(eph["ra_deg"].min(), eph["ra_deg"].max()))
print("Dec range: {:.4f} – {:.4f} deg".format(eph["dec_deg"].min(), eph["dec_deg"].max()))

# Search polygons built from the ephemeris
print("\nSearch polygons:", len(pipeline.polygons))

# Kept cutouts and their metadata
print("\nKept frames:", len(pipeline.cutouts))
for meta in pipeline.frame_metadata:
    print(f"  {meta['band']}-band  {meta['time'].utc.iso[:16]}")